### charger les docs csv

attention bug

In [42]:
import sqlite3
import pandas as pd
import os

data_folder = './data/'
csv_folder = os.path.join(data_folder, 'csv')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

for file in csv_files:
    filename = os.path.join(csv_folder, file)
    sqlite_filename = os.path.join(sql_folder, file[:-3] + "sqlite3")

    con = sqlite3.connect(sqlite_filename)

    if file == "Sondage_LV2.csv":
        pd.read_csv(filename, encoding='latin1', usecols=['Nom','mail','Langues']).to_sql(file[:-4], con, if_exists='replace', index=False)
    else:
        pd.read_csv(filename, encoding='latin1', usecols=['Nom','Mail','Note/10']).to_sql(file[:-4], con, if_exists='replace', index=False)

    con.close()


In [8]:
import csv
import json

# Chemin vers le fichier CSV d'entrée
input_csv = "data/data_students/Info_student.csv"

# Chemin vers le fichier JSON de sortie
output_json = "data/data_students/Info_student.json"

# Fonction pour lire le fichier CSV et le convertir en JSON
def csv_to_json(csv_file, json_file):
    # Ouvrir le fichier CSV en mode lecture
    with open(csv_file, "r", newline="") as file:
        # Lire le contenu du fichier CSV
        csv_data = csv.DictReader(file)
        # Convertir les données du CSV en une liste de dictionnaires
        data_list = [row for row in csv_data]
    
    # Convertir la liste de dictionnaires en JSON
    json_data = json.dumps(data_list, indent=4)
    
    # Écrire le JSON dans un fichier
    with open(json_file, "w") as json_file:
        json_file.write(json_data)

# Appel de la fonction pour effectuer la conversion
csv_to_json(input_csv, output_json)


### charger les docs json

je ne sais plus ce que fait le premier mais il faut utiliser le deuxième

In [54]:
import sqlite3
import pandas as pd
import os
import json

data_folder = './data/'
json_folder = os.path.join(data_folder, 'json')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

for file in json_files:
    filename = os.path.join(json_folder, file)
    sqlite_filename = os.path.join(sql_folder, file[:-4] + "sqlite3")
    con = sqlite3.connect(sqlite_filename)
    with open(filename, 'r', encoding='utf-8-sig') as json_file:
        data = json.load(json_file)
        if file == "Sondage_LV2.json":
            pd.DataFrame(data, columns=['Nom','Prénom','mail','Langues']).to_sql(file[:-5], con, if_exists='replace', index=False)
        else:
            pd.DataFrame(data, columns=['Nom','Prénom','mail','Note/10']).to_sql(file[:-5], con, if_exists='replace', index=False)

    con.close()


In [46]:
import sqlite3
import pandas as pd
import os
import json
import numpy as np

data_folder = './data/'
json_folder = os.path.join(data_folder, 'json')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

db_path = os.path.join(sql_folder, 'data_brute.sqlite3')
conn = sqlite3.connect(db_path)

json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

for file in json_files:
    table_name = os.path.splitext(file)[0]
    json_file_path = os.path.join(json_folder, file)

    with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
        data = json.load(json_file)
        
        if file == "Sondage_LV2.json":
            df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Langues'])
        else:
            df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Note/10'])
            df['Note/10'] = df['Note/10'].replace('', np.nan)
            df['Note/10'] = df['Note/10'].astype(float)

        df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


In [19]:
import sqlite3
import pandas as pd
import os
import json
import numpy as np

data_folder = './data/'
json_folder = os.path.join(data_folder, 'data')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

db_path = os.path.join(sql_folder, 'data.sqlite3')
conn = sqlite3.connect(db_path)

json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

for file in json_files:
    table_name = os.path.splitext(file)[0]
    json_file_path = os.path.join(json_folder, file)

    with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
        data = json.load(json_file)
        df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Class'])
        df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


In [30]:
import sqlite3
import pandas as pd
import os
import json

data_folder = './data/'
json_folder = os.path.join(data_folder, 'data_students')
sql_folder = os.path.join(data_folder, 'SQL')
chemin_fichier_json = './data/json/Sondage_LV2.json'


def file_General_table(json_folder, sql_folder):
    if not os.path.exists(sql_folder):
        os.makedirs(sql_folder)

    json_to_db_column_mapping = {
    'Nom': 'NAME',
    'Prénom': 'SURNAME',
    'mail': 'EMAIL',
    'Class': 'CLASS' 
    }

    db_path = os.path.join(sql_folder, 'data.sqlite3')
    conn = sqlite3.connect(db_path)

    json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]
    desired_table_name = 'General'

    for file in json_files:
        json_file_path = os.path.join(json_folder, file)

        with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
            data = json.load(json_file)

            # Transform the data based on the defined mapping
            transformed_data = []
            for item in data:
                transformed_item = {db_column: item.get(json_field) for json_field, db_column in json_to_db_column_mapping.items()}
                transformed_data.append(transformed_item)

            df = pd.DataFrame(transformed_data)
            df.to_sql(desired_table_name, conn, if_exists='append', index=False)

    conn.close()
    return

def complete_General_table_lv2(sql_folder, chemin_fichier_json):
    with open(chemin_fichier_json, 'r', encoding='utf-8-sig') as fichier:
        donnees = json.load(fichier)
        db_path = os.path.join(sql_folder, 'data.sqlite3')
        conn_destination = sqlite3.connect(db_path)
        cursor_destination = conn_destination.cursor()

        for ligne in donnees:

            #print(ligne["Nom"], ligne["Prénom"], ligne["mail"], ligne["Langues"] )
            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne["Nom"], ligne["Prénom"], ligne["mail"]))
            count = cursor_destination.fetchone()[0]
            
            if count == 0:
                print(ligne, "doesn't exist")
            else :
                cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne["Langues"],"PRESENT",ligne["Nom"], ligne["Prénom"], ligne["mail"]))

        conn_destination.commit()
        conn_destination.close()


file_General_table(json_folder, sql_folder)
complete_General_table_lv2(sql_folder, chemin_fichier_json)


### charger les docs xlsx

In [2]:
import sqlite3
import pandas as pd
import os

data_folder = './data/'
xlsx_folder = os.path.join(data_folder, 'xlsx')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

xlsx_files = [f for f in os.listdir(xlsx_folder) if f.endswith('.xlsx')]

for file in xlsx_files:
    filename = os.path.join(xlsx_folder, file)
    sqlite_filename = os.path.join(sql_folder, file[:-4] + "sqlite3")

    con = sqlite3.connect(sqlite_filename)

    if file == "Sondage_LV2.xlsx":
        pd.read_excel(filename, usecols=['Nom','Prénom','mail','Langues']).to_sql(file[:-5], con, if_exists='replace', index=False)
    else:
        pd.read_excel(filename, usecols=['Nom','Prénom','Mail','Note/10']).to_sql(file[:-5], con, if_exists='replace', index=False)

    con.close()


### Charge any file 

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import json
import os

def load_survey(depot_note_folder,sql_folder,name_server_brute ):
    if not os.path.exists(sql_folder):
        os.makedirs(sql_folder)

    db_path = os.path.join(sql_folder, name_server_brute)
    conn = sqlite3.connect(db_path)

    for file in [f for f in os.listdir(depot_note_folder)]:
        file_path = os.path.join(depot_note_folder, file)
        if file.endswith('.csv'):
            filename = os.path.join(depot_note_folder, file)
            if file == "Sondage_LV2.csv":
                df = pd.read_csv(filename, encoding='utf8', usecols=['Nom','mail','Langues'])
            else:
                df = pd.read_csv(filename, encoding='utf8', usecols=['Nom','Mail','Note/10'])
                df['Note/10'] = df['Note/10'].replace('', np.nan)
                df['Note/10'] = df['Note/10'].astype(float)
            df.to_sql(file[:-4], conn, if_exists='replace', index=False)
            
        elif file.endswith('.json'):
            json_file_path = os.path.join(depot_note_folder, file)
            with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
                data = json.load(json_file)
                if file == "Sondage_LV2.json":
                    df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Langues'])
                else:
                    df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Note/10'])
                    df['Note/10'] = df['Note/10'].replace('', np.nan)
                    df['Note/10'] = df['Note/10'].astype(float)
                df.to_sql(file[:-5], conn, if_exists='replace', index=False)
            
        elif file.endswith('.xlsx'):
            filename = os.path.join(depot_note_folder, file)
            if file == "Sondage_LV2.xlsx":
                df = pd.read_excel(filename, usecols=['Nom','Prénom','mail','Langues'])
            else :
                df = pd.read_excel(filename, usecols=['Nom','Prénom','Mail','Note/10'])
                df['Note/10'] = df['Note/10'].replace('', np.nan)
                df['Note/10'] = df['Note/10'].astype(float)
            df.to_sql(file[:-5], conn, if_exists='replace', index=False)
        
        else:
            print(f"Format de fichier non pris en charge: {file}")

    conn.close()
    return



data_folder = './data/'
depot_note_folder = os.path.join(data_folder, 'depot')
sql_folder = os.path.join(data_folder, 'SQL')
name_server_brute = 'data_brute.sqlite3' 

load_survey(depot_note_folder,sql_folder,name_server_brute)

### Find table names

entry: chemin de la database

In [25]:
import sqlite3

def list_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'data/SQL/data.sqlite3'
print("Tables in the database:", list_tables(Data))

Tables in the database: ['General', 'Teachers', 'Availabilities', 'Rooms', 'Summary', 'Availability_Teachers', 'Availability_Rooms']


In [45]:
import sqlite3

def list_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'data/SQL/data_brute.sqlite3'
print("Tables in the database:", list_tables(Data))

Tables in the database: ['Anglais', 'Espagnol', 'Sondage_LV2', 'Allemand', 'Info_student']


### Delete table

!!!! attention c'est définitif !!!!!

In [27]:
import sqlite3

Data_brute = 'data/SQL/data_brute.sqlite3'
def Delete_all_tables(filename):
    for i in list_tables(filename):
        conn = sqlite3.connect(filename)
        cursor = conn.cursor()
        if i != 'General':
            cursor.execute("DROP TABLE " + i + ";")
        tables = cursor.fetchall()
        conn.close()
    return [table[0] for table in tables]

#Delete_all_tables(Data_brute)

def Delete_one_table(filename, name_table):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE " + name_table + ";")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

#Delete_one_table(Data_brute, "")

Data = 'data/SQL/data.sqlite3'


def Delete_data_table(filename, name_table):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM " + name_table + ";")
    conn.commit() 
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

#Delete_data_table(Data,"'General'")

### create one general table & other tables also

entry: chemin de la database

In [28]:
import sqlite3

def Create_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE General (NAME VARCHAR(50),SURNAME VARCHAR(50),EMAIL VARCHAR(50),CLASS VARCHAR(5),LV1 VARCHAR(50),GRADE_LV1 INT,GROUP_LV1 INT,LV2 VARCHAR(50),GRADE_LV2 INT,GROUP_LV2 VARCHAR(50), EXTRA_TIME BOOLEAN, STATUS VARCHAR(50));")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'data/SQL/data.sqlite3'
print("Tables in the database:", Create_tables(Data))

Tables in the database: []


In [5]:
import sqlite3


def Create_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE Teachers  (ID_teacher INT, NAME VARCHAR(50),SURNAME VARCHAR(50),EMAIL VARCHAR(50), SUBJECT VARCHAR(20));")        # create teacher table
    cursor.execute("CREATE TABLE Availabilities  (ID_Availability INT, Day VARCHAR(12),Hour VARCHAR(5));")                                         # create availabilities table
    cursor.execute("CREATE TABLE Rooms  (ID_teacher INT, NAME VARCHAR(50),SURNAME VARCHAR(50),EMAIL VARCHAR(50), SUBJECT VARCHAR(20));")           # create Rooms table
    cursor.execute("CREATE TABLE Summary  (ID_Group VARCHAR(10), ID_teacher INT, ID_Room INT,ID_Availability INT);")                               # create teacher table
    cursor.execute("CREATE TABLE Availability_Teachers  (ID_Reacher INT, ID_Availability INT);")                                                   # create Availability_Teachers table
    cursor.execute("CREATE TABLE Availability_Rooms  (ID_Room INT, ID_Availability INT);")                                                         # create Availability_Rooms table
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'data/SQL/data.sqlite3'
print("Tables in the database:", Create_tables(Data))

Tables in the database: []


### file general table & other tables also

A faire avec les data_student

In [ ]:
# for table in the same database
import sqlite3

def Create_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO General (NAME,SURNAME,EMAIL,CLASS) SELECT colonne_a_copier1, colonne_a_copier2, ..., colonne_a_copierN FROM Sondage_LV2;")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'data/SQL/data.sqlite3'
#print("Tables in the database:", Create_tables(Data))

In [10]:
import sqlite3
## charger les data eleves from Info_student
def copy_data_between_databases(source_db, destination_db):
    # Connexion à la base de données source
    conn_source = sqlite3.connect(source_db)
    cursor_source = conn_source.cursor()
    
    # Connexion à la base de données destination
    conn_destination = sqlite3.connect(destination_db)
    cursor_destination = conn_destination.cursor()
    
    # Exécution de la requête pour copier les données entre les bases de données
    cursor_source.execute("SELECT Nom, Prénom, mail, Class FROM Info_student;")
    data_to_copy = cursor_source.fetchall()
    
    for row in data_to_copy:
        cursor_destination.execute("INSERT INTO General (NAME, SURNAME, EMAIL, CLASS, EXTRA_TIME, STATUS) VALUES (?, ?, ?, ?, FALSE, 'PRESENT' );", row)
    
    conn_destination.commit()
    
    # Fermeture des connexions
    conn_source.close()
    conn_destination.close()

# Exemple d'utilisation
source_database = 'data/SQL/data_brute.sqlite3'
destination_database = 'data/SQL/data.sqlite3'
copy_data_between_databases(source_database, destination_database)


In [29]:
import sqlite3
# a revoir 

def copy_data_between_databases(source_db, destination_db):
    # Connexion à la base de données source
    conn_source = sqlite3.connect(source_db)
    cursor_source = conn_source.cursor()
    
    # Connexion à la base de données destination
    conn_destination = sqlite3.connect(destination_db)
    cursor_destination = conn_destination.cursor()
    
    # Exécution de la requête pour copier les données entre les bases de données
    cursor_source.execute("SELECT Nom, Prénom, mail, `Note/10` FROM Anglais;")
    data_to_copy = cursor_source.fetchall()

    for row in data_to_copy:
        # Check if the data already exists based on name, surname, or email
        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE NAME = ? AND SURNAME = ? OR EMAIL = ?;", (row[0], row[1], row[2]))
        count = cursor_destination.fetchone()[0]
        
        if count == 0:
            # If data does not exist, insert it into the destination table
            #cursor_destination.execute("INSERT INTO General (LV1, GRADE_LV1, NAME, SURNAME, EMAIL) VALUES (?, ?, ?, ?, ?);", ('English', row[3], row[0], row[1], row[2]))
            print(row, "doesn't exist")
        else :
            cursor_destination.execute("UPDATE General SET LV1=?, GRADE_LV1=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", ('English', row[3], row[0], row[1], row[2]))


    conn_destination.commit()
    
    # Fermeture des connexions
    conn_source.close()
    conn_destination.close()

# Exemple d'utilisation
source_database = 'data/SQL/data_brute.sqlite3'
destination_database = 'data/SQL/data.sqlite3'
copy_data_between_databases(source_database, destination_database)


### add forecast

entry: nb of forecast, database, table(subject), mu, sigma

In [10]:
import sqlite3
import numpy as np

forecast = 30
Data = 'data/SQL/data_brute.sqlite3'
table= 'Anglais'

def parameters(filename,table_add):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT `Note/10` FROM " + table_add)
    notes = cursor.fetchall()
    conn.commit()
    conn.close()
    numeric_values = [note[0] for note in notes if isinstance(note[0], (int, float))]
    mu = np.mean(numeric_values)
    sigma = np.var(numeric_values)**0.5
    return  mu, sigma

def add_forcast(filename,table_add, nb_forecast, mu , sigma ):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT 'Note/10' FROM " + table_add)
    note = cursor.fetchall()
    conn.commit()
    conn.close()
    for i in range(0,nb_forecast):
        conn = sqlite3.connect(filename)
        cursor = conn.cursor()
        valeur_gaussienne = str(np.random.uniform(mu, sigma))
        cursor.execute("INSERT INTO " + table_add + " ('Nom', 'Prénom', 'mail', 'Note/10' ) VALUES ('forecast', 'forecast', 'forecast','" + valeur_gaussienne + "')")
        group = cursor.fetchall()
        conn.commit()
        conn.close()
    return group 

print(forecast,"forecast add", add_forcast(Data,table,forecast,parameters(Data, table)[0], parameters(Data, table)[1] ))


30 forecast add []


### Create groups

entry: database, table (subject), nb by class

enregistrer les groupes dans database general

In [39]:
import sqlite3

def extract_group(filename, table_add):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM " + table_add + " ORDER BY [Note/10] DESC")
    group = cursor.fetchall()
    conn.close()
    return group


Data = 'data/SQL/data_brute.sqlite3'
table= 'Anglais'
group_data = extract_group(Data, table)


def nomber_class(nomber_student, nomber_by_class):
    nomber_class = 0
    # à expliquer 
    if (nomber_student % nomber_by_class) > ((nomber_student //nomber_by_class)*(2/3)) :
        nomber_class = nomber_student//nomber_by_class + 1
    else :
        nomber_class = nomber_student//nomber_by_class
    return nomber_class

nb_student = len(group_data)
nb_by_class = 16
nb_class = nomber_class(nb_student,nb_by_class)


def groups(list_student, number_class):
    
    # Step 2: Divide the sorted list of students into groups
    total_students = len(list_student)
    students_per_group = total_students // number_class
    extra_students = total_students % number_class
    groups = []
    start_idx = 0
    for i in range(number_class):
        end_idx = start_idx + students_per_group
        if i < extra_students//2 :
            end_idx = end_idx
        elif i <  extra_students +  extra_students//2:  # If there are remaining students, distribute them
            end_idx += 1
        groups.append(list_student[start_idx:end_idx])
        start_idx = end_idx
    return groups

groups = groups(group_data, nb_class)
#print(groups)

#i = 1
#for group in groups:
    #print('\nGroupe\n', len(group), "\n", group)
    #mean = 0
    #for student in group:
    #    mean =+ mean + float(student[3])
    #print(len(group))
    #print('Mean', mean/len(group))
    #    print('Group ' + str(i), student)
    #i+=1


In [40]:

import sqlite3
# a revoir 

def copy_data_between_databases(groups, destination_db):
    
    # Connexion à la base de données destination
    conn_destination = sqlite3.connect(destination_db)
    cursor_destination = conn_destination.cursor()
    i = 1
    for group in groups :
        name_group = 'Group ' + str(i)

        for student in group:
            # Check if the data already exists based on name, surname, or email
            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE NAME = ? AND SURNAME = ? OR EMAIL = ?;", (student[0], student[1], student[2]))
            count = cursor_destination.fetchone()[0]
            
            if count == 0:
                # If data does not exist, insert it into the destination table
                print(student, "doesn't exist")
            else :
                cursor_destination.execute("UPDATE General SET GROUP_LV1=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (name_group, student[0], student[1], student[2]))
        i +=1

    conn_destination.commit()
    
    # Fermeture des connexions
    conn_destination.close()
# Exemple d'utilisation
destination_database = 'data/SQL/data.sqlite3'
copy_data_between_databases(groups, destination_database)

### export the group in PDF 

entry : liste des groupes + matière du groupe

generalisation a finaliser

In [25]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER
import os


def pdf_maker(groups, langage):
    i = 1
    styles = getSampleStyleSheet()  

    for group in groups:
        pdf_path = f"data/pdf/{langage}_Group_{i}.pdf"
        if os.path.exists(pdf_path):
            os.remove(pdf_path)

        doc = SimpleDocTemplate(pdf_path, pagesize=letter)
        content = []

        title = Paragraph(f"<b>{langage}: Group {i}</b>", styles["Title"])
        content.append(title)
        content.append(Spacer(1, 12))  

        data = group
        table_data = [("Name", "Surname", "Email", "Mark")]
        table_data.extend(data)  

        table = Table(table_data)
        style = TableStyle([('BACKGROUND', (0,0), (-1,0), colors.grey),
                            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
                            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
                            ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
                            ('BOTTOMPADDING', (0,0), (-1,0), 12),
                            ('BACKGROUND', (0,1), (-1,-1), colors.beige),
                            ('GRID', (0,0), (-1,-1), 1, colors.black)])
        table.setStyle(style)

        content.append(table)

        content.append(Spacer(1, 12)) 
        centered_style = ParagraphStyle(name='Centered', parent=styles['Normal'], alignment=TA_CENTER)

        marks = [member[-1] for member in group]  # Supposer que la note est le dernier élément de chaque tuple
        mean_mark = sum(marks) / len(marks) if marks else 0

        summary = Paragraph(f"Number of student: {len(group)} ; Mark mean: {mean_mark:.2f}", centered_style)
        content.append(summary)
        doc.build(content)
        #print(f"Le fichier PDF '{pdf_path}' a été créé avec succès.")
        i += 1


pdf_maker(groups, "Anglais")
